In [6]:
import os
import time
import torch
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoModelForCausalLM, AutoTokenizer

In [7]:
checkpoint = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [8]:
class FRIDAY():
    # initialize
    def __init__(self):
        # once chat starts, the history will be stored for chat community
        self.chat_history_ids = None
        # make input ids global to use them anywhere within the object
        self.bot_input_ids = None
        # a flag to check whether to end the conversation
        self.end_chat = False
        # greet while stating
        self.welcome()
        
    def welcome(self):
        print("Initiating ChatBot...")
        # some time for the user to get ready
        time.sleep(2)
        print('Type "bye" or "quit" or "exis" to end the chat \n')
        # give them time to read what is printed
        time.sleep(3)
        # greet and introduce
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot>> " + greeting)
        
    def user_input(self):
        text = input("User>> ")
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            self.end_chat = True
            print('ChatBot>> See you soon. Bye!')
            time.sleep(1)
            print('\nQuitting ChatBot...')
        else:
            self.new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, \
                return_tensors='pt')
            
    def bot_response(self):
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=1)
        else:
            self.bot_input_ids = self.new_user_input_ids
            
            # define the new chat history ids based on the preceding chats
            self.chat_history_ids = model.generate(self.bot_input_ids, max_length=1000, \
                pad_token_id=tokenizer.eos_token_id)
            # last ouput tokens from bot
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        # in case, bot fails to answer
        if response == "":
            response = self.random_response()
        # print bot response
        print('ChatBot >>  '+ response)
    
     # in case there is no response from model
    def random_response(self):
        i = -1
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # iterate over history backwards to find the last token
        while response == '':
            i = i-1
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # if it is a question, answer suitably
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", 
                                     "I am not sure"])
        # not a question? answer suitably
        else:
            reply = np.random.choice(["Great", 
                                      "Fine. What's up?", 
                                      "Okay"
                                     ])
            
        return reply

In [9]:
# build a ChatBot object
bot = FRIDAY()
# start chatting
while True:
    # receive user input
    bot.user_input()
    # check whether to end chat
    if bot.end_chat:
        break
    # output bot response
    bot.bot_response()

Initiating ChatBot...
Type "bye" or "quit" or "exis" to end the chat 

ChatBot>> Hey, Great day! I am your virtual assistant


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


ChatBot >>  Hello! :D
ChatBot >>  Okay
ChatBot>> See you soon. Bye!

Quitting ChatBot...
